# Fritz's notebook: Automatic measurements with amie 

Amie integrates fully in an automated experimental setup. Register through the (app.amie.ai) and get your API keys under 'setup' to log in. The garden contains all data that is accessible to you, so all your trees and leaves and the ones that are shared with you through a team. 

As with ```calibrate``` you can integrate documentation directly into the measurements functions. Fritz wrote the calibrate function to automatically call an instrument driver and then upload the parameteres and the calibration curves to amie.

In [ ]:
import amieci
import random
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt

def calibrate(leaf, instrument):
    calibration, plot = instrument.calibrate()
    leaf.kvs.add_dict(calibration)
    leaf.add_plot(calibration['instrument_name'] + '.png', 
                  plot, 
                  caption = calibration['instrument_name'] + '_calibration')
    return leaf

url = "https://9453a694-bd17-412a-9c9b-staging.app.amie.ai/api"
url_local = "http://localhost:8080/api"

garden = amieci.Garden("aaaaaa", "fritz@wine.com", url)
garden.load()

## Day 1
Here Fritz makes a new tree and adds a title and comment right in the notebook. He can change them later in the app.
```garden.ct.```is the current tree.

In [ ]:
tree = garden.new_tree()
garden.ct.set_title("Chemical analysis of ranked wines")
garden.ct.set_description("""I will first store the wine in the cellar with controlled temperature and humidity.
                Both are read via an IoT device connected to amie. I measure alcohol content, sulfur, acidity and sugar. 
                First I have to calibrate all of them because they are probably wildly out of spec right now.""")
garden.ct.save()

Since the cellar IoT device is metadata in the experiments, Fritz adds the dictionary it returns as key value pairs to the leaf with ```.kvs.add_dict```.
To have easy access to the leaf, he stores it in a variable with ```day1_leaf = garden.ct.cl```.

In [ ]:
from instruments import CellarIoT

garden.ct.new_leaf()
garden.ct.cl.set_title("The condition in the cellar")
garden.ct.cl.set_description("""The conditions in the cellar are tracked with and IoT device which gets periodically checked and read out when I call it
                                I add the conditions every time I save data from now on.""")
parameters = CellarIoT.measure()
garden.ct.cl.kvs.add_dict(parameters)
garden.ct.cl.save()
day1_leaf = garden.ct.cl

## Day 2
Fritz runs 4 Calibration runs on the 4 instruments (connected to amie). Every measurement combines data from the instrument and the IoT device. ```calibrate()``` automatically adds data and the generated plots to amie.

In [ ]:
from instruments import AlcoholTester, AcidTester, SulfurTester, SugarTester

In [ ]:
alc = garden.ct.new_leaf(parent=day1_leaf)
alc.set_title("Alcohol calibration")
alc.set_description("A calibration run before we can test the real deal")
alc = calibrate(alc, AlcoholTester)
alc.save()

acid = garden.ct.new_leaf(parent=day1_leaf)
acid.set_title("Acidity calibration")
acid.set_description("A calibration run before we can test the real deal")
acid = calibrate(acid, AcidTester)
acid.save()

sulfur = garden.ct.new_leaf(parent=day1_leaf)
sulfur.set_title("Sulfur calibration")
sulfur.set_description("A calibration run before we can test the real deal")
sulfur = calibrate(sulfur, SulfurTester)
sulfur.save()

sugar = garden.ct.new_leaf(parent=day1_leaf)
sugar.set_title("Sugar calibration")
sugar.set_description("A calibration run before we can test the real deal")
sugar = calibrate(sugar, SugarTester)
sugar.save()

## Day 3
Fritz sets up a longer measurement. Data, plots, metadata and his comments are automatically added to amie. Once the measurement runs the can check in the app, from home or on his phone how things are going. He sets it up to run completely from the python client, but he could also add his comments and thoughts in the app later.

In [ ]:
parameters = {'frequency':'9' ,'sensitivity':'13', 'automatic' : 'True'}
data, fig = AlcoholTester.measure(parameters)
alc = garden.ct.new_leaf(parent=alc)
alc.set_title("Alcohol measurement")
alc.set_description("""Testing Francesca's wine samples for their alcohol content.  Testing with #instrument_name acli_top_100# a hygrometer:\n
                        1. Clean all glassware; cylinder and mixing beacons
                        2. Heat to #temperature 20#.
                        3. Set #automatic True#, because it's well calibrated.
                        4. Pour 100 ml and let stand for 20 min
                        5. Set #mass_constant 30#
                        6. Adjust #flow 231#
                        7. Load the sample, press start
                        8. Done! \n
                        See $Img 1$ for the plots and $Data 2$ for the file.""")
alc.kvs.add_dict(CellarIoT.measure())
alc.kvs.add_dict(parameters)
alc.add_plot('alcohol.png', fig, caption = 'Alcohol measurement')
alc.add_data('alcohol.csv', bytes(data, 'utf-8'), caption = 'Alcohol measurement')
alc.save()

parameters = {'frequency':'13' ,'sensitivity':'30', 'automatic' : 'True'}
data, fig = AcidTester.measure(parameters)
acid = garden.ct.new_leaf(parent=acid)
acid.set_title("Acidity measurement")
acid.set_description("""Testing Francesca's wine samples for their acidity. We will test for fixed acidity,
                        volatile acidity, citric acids and pH with #instrument_name sweet_and_sour-334# :\n
                        1. Clean all the glassware
                        2. Dilute the samples with distilled water.
                        3. Start pumping down.
                        4. Set #automatic True#, because it's well calibrated.
                        5. Once the setup is pumped down, start the heater.
                        6. Spin at #rate 3000#
                        7. 5nmofNi of Ge XXXV nm sollicitatur
                        9. Load the sample
                        10. Done! \n
                        See $Img 1$ for the plots and $Data 2$ for the file.""")
acid.kvs.add_dict(CellarIoT.measure())
acid.kvs.add_dict(parameters)
acid.add_plot('acid.png', fig, caption = 'Acidity measurement')
acid.add_data('acid.csv', bytes(data, 'utf-8'), caption = 'Acidity measurement')
acid.save()

parameters = {'frequency': 70 ,'sensitivity': 10, 'automatic' : 'True', 'temperature_offset' : 0}
data, fig = SulfurTester.measure(parameters)
sulfur = garden.ct.new_leaf(parent=sulfur)
sulfur.set_title("Sulfur measurement")
sulfur.set_description("""Testing Francesca's wine samples for their sulfur content.  Testing with #instrument_name sulfi-20#.
                        We teset for free sulfur dioxide, totla sulfur and sulphates\n
                        1. Clean all glassware.
                        2. Cool to #temperature 10#.
                        3. Set #mass_constant 30#
                        4. Set #automatic True#, and #temperature_offset 0#, because it's well calibrated.
                        4. Pour 100 ml and let stand for 20 min \n
                        5. Set #sensitivity 10#.
                        6. Set #resolution 20#, there is plenty of sample and it shouldn't take all night. 
                        6. Adjust #flow 231#. 
                        7. Load the sample, press start.
                        8. Done! \n
                        See $Img 1$ for the plots and $Data 2$ for the file.""")
sulfur.kvs.add_dict(CellarIoT.measure())
sulfur.kvs.add_dict(parameters)
sulfur.add_plot('sulfur.png', fig, caption='Sulfur mesurement')
sulfur.add_data('sulfur.csv', bytes(data, 'utf-8'), caption = 'Sulfur measurement')
sulfur.save()

parameters = {'frequency':'43' ,'sensitivity':'9', 'automatic' : 'True'}
data, fig = SugarTester.measure(parameters)
sugar = garden.ct.new_leaf(parent=sugar)
sugar.set_title("Sugar measurement")
sugar.set_description("""Testing Francesca's wine samples for their sugar content.  Testing with #instrument_name KK-20#.
                        We test for residual sugar, total sulfur, and sulfates. \n
                        1. Clean all glassware;
                        2. Cool to #temperature 10#.
                        3. Set #mass_constant 30#
                        4. Set #automatic True# because it's well calibrated.
                        5. Adjust #inlet_pressure 1040# and let it stand for a while.
                        6. Set #frequency 43#
                        6. Adjust #flow 231#
                        7. Check for #outlet_pressure 330#.
                        8. Load the sample and press start. Done! \n
                        See $Img 1$ for the plots and $Data 2$ for the file.""")
sugar.kvs.add_dict(CellarIoT.measure())
sugar.kvs.add_dict(parameters)
sugar.add_plot('sugar.png', fig, caption = 'Sugar measurement')
sugar.add_data('sugar.csv', bytes(data, 'utf-8'), caption = 'Sugar measurement')
sugar.save()

## Day 20 
Some sample 🍷 is gone, and it turns out that there has been a mistake. Fritz can see in data scientist Anders' analysis in the app why there might have been a mistake. He identifies the mistake by plotting his previous measurement parameters right in the app and spots that there might have been a problem with the cellar temperature in one of the measurements.

In [ ]:
import amieci
import random
import pandas as pd 
%reload_ext autoreload
%autoreload 2
garden = amieci.Garden("aaaaaa", "fritz@wine.com", url)
garden.load()

He logs in again and loads his garden to pick up from where he has left off. To add the re-done measurement to the right leaf, he grabs its ID from the app and loads it into a variable.

In [ ]:
garden.set_cl('79a9f727-909e-4429-a29c-5e4e366ccdae')
sulfur = garden.ct.cl

In [ ]:
parameters = {'frequency':'43' ,'sensitivity':'9', 'automatic' : 'True', 'temperature_offset' : 'auto',
             'resolution' : '60'}
data, fig = SulfurTester.measure(parameters)
sulfur = garden.ct.new_leaf(parent=sulfur)
sulfur.set_title("Re-do the sulfur measurement")
sulfur.set_description("""I have to re-test for sulfur. Luckily there is still some 🍷 left. See $Img 1$ for the plots and $Data 2$ for the file.
                        I found some cellar temperature irregularities which most likely messed everything up.
                        Again Testing with #instrument_name sulfi-20#. This time with #resolution 60# and #temperature_offset auto# to make sure it compensates for a change since calibration.
                        We tested for free sulfur dioxide, total sulfur and sulfates.
                        1. Clean all glassware.
                        2. Cool to #temperature 10#.
                        3. Set #mass_constant 30#
                        4. Set #automatic True#, and #temperature_offset auto#, because it's maybe not so well calibrated.
                        4. Pour 100 ml and let stand for 20 min.
                        5. Set #sensitivity 10#.
                        6. Set #resolution 60# to get it right this time. 
                        6. Adjust #flow 231#. 
                        7. Load the sample, press start.
                        8. Done!""")

sulfur.kvs.add_dict(CellarIoT.measure())
sulfur.kvs.add_dict(parameters)
sulfur.add_plot('sulfur.png', fig, caption='Sulfur mesurement')
sulfur.add_data('new_sulfur.csv', bytes(data, 'utf-8'), caption='new_sulfur')
sulfur.save()
sulfur = garden.ct.cl